In [1]:
import json
import pandas as pd
from tqdm import tqdm
 

In [2]:
beir_data_folder = '/drive02/novak/huggingface_cache/BeIR/msmarco/'
train_data_filepath = '/drive02/novak/datasets/qa_datasets/msmarco-query_passage_negative_v2.jsonl'

In [5]:
## procitaj train q and p
with open(train_data_filepath) as rf:
    
    train_data = [json.loads(line) for line in tqdm(rf.readlines())]

    train_queries = [x['query'] for x in tqdm(train_data)]
    
    train_pos = [x['pos'] for x in tqdm(train_data)]


100%|██████████| 17579773/17579773 [00:11<00:00, 1518921.90it/s]


In [7]:
## procitaj test q and p
with open(f'{beir_data_folder}queries.jsonl') as rf:
    
    test_data_q = [json.loads(line) for line in tqdm(rf.readlines())]

    test_queries = [x['text'] for x in tqdm(test_data_q)]
    
with open(f'{beir_data_folder}corpus.jsonl') as rf:
    
    test_data_p = [json.loads(line) for line in tqdm(rf.readlines())]

    test_pos = [x['text'] for x in tqdm(test_data_p)]

100%|██████████| 8841823/8841823 [00:06<00:00, 1384288.13it/s]


In [12]:
## procitaj relacije train,dev,test
# with open(f'{beir_data_folder}qrels/test.tsv') as rf:
test_rel = pd.read_csv(f'{beir_data_folder}qrels/test.tsv', sep='\t')
dev_rel = pd.read_csv(f'{beir_data_folder}qrels/dev.tsv', sep='\t')
train_rel = pd.read_csv(f'{beir_data_folder}qrels/train.tsv', sep='\t')

In [13]:
## kreiraj dict za id-to-text

id_to_q = {data['_id']:data['text'] for data in tqdm(test_data_q)}
id_to_p = {data['_id']:data['text'] for data in tqdm(test_data_p)}

100%|██████████| 8841823/8841823 [00:13<00:00, 657466.72it/s]


In [37]:
real_test_q_ids_set = set(test_rel[test_rel['score']>0]['query-id'].tolist())
real_test_p_ids_set = set(test_rel[test_rel['score']>0]['corpus-id'].tolist())

real_dev_q_ids_set = set(dev_rel[dev_rel['score']>0]['query-id'].tolist())
real_dev_p_ids_set = set(dev_rel[dev_rel['score']>0]['corpus-id'].tolist())

real_train_q_ids_set = set(train_rel[train_rel['score']>0]['query-id'].tolist())
real_train_p_ids_set = set(train_rel[train_rel['score']>0]['corpus-id'].tolist())

In [39]:
real_test_q = [id_to_q[str(i)] for i in real_test_q_ids_set]
real_test_p = [id_to_p[str(i)] for i in real_test_p_ids_set]

real_dev_q = [id_to_q[str(i)] for i in real_dev_q_ids_set]
real_dev_p = [id_to_p[str(i)] for i in real_dev_p_ids_set]

real_train_q = [id_to_q[str(i)] for i in real_train_q_ids_set]
real_train_p = [id_to_p[str(i)] for i in real_train_p_ids_set]

print(len(real_test_q), len(real_test_p))
print(len(real_dev_q), len(real_dev_p))
print(len(real_train_q), len(real_train_p))


43 4102
6980 7433
502939 516472


In [34]:
train_pos_set = set(train_pos)
train_queries_set = set(train_queries)

4102

In [43]:
## test q se NE nalaze u train set-u
## dev q se NE nalaze u train set-u
## svi train q se nalaze u train set-u 


## 352 od 4102 tekstova iz test set-a se nalazi u nasem train set-u(nastalo zato sto se ovi tekstovi nalaze i u train set-u)
## 306 od 7433 tekstova iz dev set-a se nalazi u nasem train set-u(nastalo zato sto se ovi tekstovi nalaze i u train set-u)
## svi tekstovi iz train set-a se nalaze u nasem train set-u

cnt = 0
for x in tqdm(real_dev_q):
    if x in train_queries_set:
        cnt+=1
print(cnt)

100%|██████████| 6980/6980 [00:00<00:00, 1180922.19it/s]

0


In [44]:
## presek tekstova izmedju train,dev,test

real_train_p_set = set(real_train_p)
real_dev_p_set = set(real_dev_p)
real_test_p_set = set(real_test_p)


In [48]:
print(f'set size dev:{len(real_dev_p_set)}')
print(f'set size train:{len(real_train_p_set)}')
print(f'set size test:{len(real_test_p_set)}')
print('*'*50)
print(f'presek dev i train:{len(real_train_p_set.intersection(real_dev_p_set))}')
print(f'presek test i train:{len(real_train_p_set.intersection(real_test_p_set))}')
print(f'presek dev i test:{len(real_test_p_set.intersection(real_dev_p_set))}')

set size dev:7433
set size train:516470
set size test:4101
**************************************************
presek dev i train:307
presek test i train:354
presek dev i test:5


In [8]:
## proveri da li se neki train q nalaze u test q 

print(f'train lens, diff q: {len(set(train_queries))}, diff p: {len(set(train_pos))}')

print(f'test lens, diff q: {len(set(test_queries))}, diff p: {len(set(test_pos))}')


train lens, diff q: 498970, diff p510865
test lens, diff q: 509962, diff p8841661
